# SemEval-2010 Task 8: Multi-Way Classification of Semantic Relations Between Pairs of Nominals

In [1]:
import tensorflow as tf
import numpy as np
from collections import Counter

## Preprocessing code here

In [20]:
import string
f=open("/home/diptarkbose/Documents/Machine Learning/Philips Hackabout/SemEval2010_task8_all_data/SemEval2010_task8_training/TRAIN_FILE.TXT", 'r')
txt1=f.read().translate(str.maketrans("\t\r", "  "))
#f.seek(0)
#words=f.read().translate(str.maketrans("\t\r", "  ")).split(" ")
#txt2=''.join([i for i in txt1 if not i.isdigit()])
txt=txt1.split('\n')
sentence_corpora=[]
sentence_labels=[]
words=[]
for i in range(0, 31984, 4):
    txt[i]=txt[i].lstrip('0123456789')
    j=txt[i].find("</e1>")
    k=txt[i].find("<e2>")
    at=str(txt[i][j+5: k].strip())
    for elem in at.split(" "):
        words.append(elem)
    sentence_corpora.append(at)
    sentence_labels.append(txt[i+1].strip().replace("(e1,e2)", "").replace("(e2,e1)", ""))

print(sentence_corpora[0:10])
print("\n")
print(sentence_labels[:10])
print("\n")
words=list(set(words))
print(words[0:10])

['of antenna', 'was carefully wrapped and bound into the', 'of a keygen uses a', 'uprises from the', '', "that is Peru's largest", 'in the distal part of the stomach caused by Helicobacter pylori', 'have been moving back into', 'was contained in a', 'was pipetted into a 25 mL glass']


['Component-Whole', 'Other', 'Instrument-Agency', 'Other', 'Member-Collection', 'Other', 'Cause-Effect', 'Entity-Destination', 'Content-Container', 'Entity-Destination']


['', 'mode', 'beer-commercial', 'invested', 'chopped', 'Ottoman', 'Waldorf', 'Russian', 'air-conditioning', 'anonymous']


In [22]:
counts = Counter(words)
vocab = sorted(counts, key=counts.get, reverse=True)
vocab_to_int = {word: ii for ii, word in enumerate(vocab, 1)}
#print (vocab_to_int)

sentence_int=[]
for sentence in sentence_corpora:
    sentence_int.append([vocab_to_int[word] for word in sentence.split()])
print(sentence_int[0:10])
print(len(sentence_int))

[[2675, 5333], [41, 1161, 402, 248, 1687, 4544, 2650], [2675, 3853, 4493, 4549, 3853], [1367, 779, 2650], [], [4957, 16, 5101, 4488], [3982, 2650, 2484, 5244, 2675, 2650, 2573, 860, 278, 1748, 234], [4387, 3048, 1034, 4819, 4544], [41, 3199, 3982, 3853], [41, 2162, 4544, 3853, 4955, 4243, 4855]]
7996


In [24]:
#Setting Label values for Softmax Classifier
label_dict={"Cause-Effect": 0, 
            "Instrument-Agency": 1, 
            "Product-Producer": 2, 
            "Content-Container": 3, 
            "Entity-Origin": 4, 
            "Entity-Destination": 5, 
            "Component-Whole": 6,
            "Member-Collection": 7,
            "Message-Topic": 8,
            "Other": 9}
final_labels=[]
for elem in sentence_labels:
    final_labels.append(label_dict[elem])
print(final_labels[0:10])
print(len(final_labels))

[6, 9, 1, 9, 7, 9, 0, 5, 3, 5]
7996


In [27]:
# Max sequnce length I could find was 10
#padding remaining spaces with 0
seq_len = 15
features = np.zeros((len(sentence_int), seq_len), dtype=int)
for i, row in enumerate(sentence_int):
    features[i, -len(row):] = np.array(row)[:seq_len]

ValueError: could not broadcast input array from shape (0) into shape (15)

In [15]:
# Hyperparams
lstm_size= 256
lstm_layers= 2
batch_size= 512 # no.of sentences fed in 1 epoch 
learning_rate=0.003

In [16]:
n_words = len(vocab_to_int)

# Create the graph object
graph = tf.Graph()
# Add nodes to the graph
with graph.as_default():
    inputs_ = tf.placeholder(tf.int32, [None, None], name='inputs')
    labels_ = tf.placeholder(tf.int32, [None, None], name='labels')
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')

In [17]:
embed_size= 256
with graph.as_default():
    embedding = tf.Variable(tf.random_uniform((n_words, embed_size), -1, 1))
    embed=tf.nn.embedding_lookup(embedding, inputs_)

In [18]:
# Main Network
lstm=tf.contrib.rnn.BasicLSTMCell(lstm_size)
drop=tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
cell=tf.contrib.rnn.MultiRNNCell([drop]*lstm_layers)
initial_state = cell.zero_state(batch_size, tf.float32)

In [19]:
# RNN Forward pass
with graph.as_default():
    outputs, final_state = tf.nn.dynamic_rnn(cell, embed, initial_state=initial_state)

ValueError: Tensor("rnn/while/LoopCond:0", shape=(), dtype=bool) must be from the same graph as Tensor("Merge:0", shape=(512, 256), dtype=float32).

In [32]:
with graph.as_default():
    predictions = tf.contrib.layers.fully_connected(outputs[:, -1], 1, activation_fn=tf.sigmoid)
    cost = tf.losses.mean_squared_error(labels_, predictions)
    
    optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

NameError: name 'outputs' is not defined